<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/hu_compariosons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule
from unidecode import unidecode

Towerdb Comparisons


In [ ]:
def find_diffs_between_files_1(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, type_file='mix', dates_cols=[], status_col='', kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        kind = kind.lower()
        if kind == 'ta':
            df = df.dropna(subset=[index_col], axis=0)
            df['sites'] = df[index_col].astype(str) + df[kind_col] 
            #df.columns = lower_str(list(df.columns))
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df = df.drop_duplicates(subset=[index_col], keep='last')
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    def date_parser(df, columns, format=1, type_dates='normal'):
        t_col = type_dates.lower()
        if format == 1:
            type_date = "%d/%m/%Y"
        else:
            type_date = "%d-%m-%Y"
        for column in lower_str(columns):
            if t_col == 'mixed':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,engine='python', encoding='latin1').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD[bill_cols]
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW, engine='python', encoding='latin1').fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW[bill_cols]
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        #cols_old = csv_header(path_OLD) header=0, names=cols_old,
        df_OLD = pd.read_csv(path_OLD, engine='python').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        date_parser(df_NEW, dates_cols, 1, 'normal')
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

hu_bill_cols = ["Active / not active",\
    "Confirmed Skylon scope",\
    "Code",\
    "Categorization by Transmission Sys",\
    "Additional information \x96 room configuration (including container information)",\
    "Active Sharing Arrangements involving the Operator",\
    "Critical site",\
    "Strategic site",\
    "Telenor tenant",\
    "Telekom tenant",\
    "Power supply for tenants",\
    "Energy category",\
    'Ready For Active Installation ("RFAI") date',\
    "Wip_Site",\
    "Bts_Site",\
    "Strategic_Site_Bucket",\
    "Critical_Site_Beyond_10",\
    "Subsequent_Sharing_Arrangement",\
    "First_Active_Sharing_Start_Date",\
    "First_Active_Sharing_End_Date",\
    "Active Sharing Deployment Types",\
    "Sites_As_Metered_Estimated",\
    "Date of equipment removal"]

path_tw = '/content/TowerDB_Hungary_20210831.csv'
path_msa = '/content/TowerDB_Hungary_20210731 (1).csv' 
tw_save = '/content/TW_HU'
new_name = 'TowerDB_Hungary_20210831.csv'
old_name = 'TowerDB_Hungary_20210731.csv'  
tw_dates = ['ready for active installation ("rfai") date',"First_Active_Sharing_Start_Date",\
            "First_Active_Sharing_End_Date", "Date of equipment removal"]
find_diffs_between_files_1(path_msa, path_tw, 'Code',hu_bill_cols, tw_save, old_name, new_name, type_file='csv',\
                           status_col='active / not active',kind='tw')

TA Comparisons

Verificar duplicatas do TA para evitar error no cross values

In [13]:
import natsort
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name, new_name, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col):
        col = lower_str(list(df.columns))
        df.columns = col
        #df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        #df = df.drop_duplicates(subset=[index_col], keep='last')
        #df[index_col] = df[index_col].astype(str)
        #df = df.iloc[natsort.index_humansorted(df[index_col])]
        for i in ['tenant (name/id)', 'mno classification']:
            df[i] = [str(j).lower() for j in  df[i]]

        df = df.sort_values(by=[index_col, 'tenant (name/id)'])
        df['sites'] = df[index_col].astype(str)+df['tenant (name/id)'].astype(str) + df['mno classification'].astype(str)
        df = df.sort_values(by=['sites'])
        """lista=[]
        dico = {}
        for id in df[index_col]:
            nome = str(id)
            if nome not in dico.keys():
                dico[nome] = 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)
            else:
                dico[nome] += 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)
        print(lista)
        df['sites'] = lista"""
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    #status_col = status_col.lower()
    
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD,index_col)

        df_NEW = pd.read_csv(path_NEW, encoding='latin1').fillna('')
        df_NEW = fit_df(df_NEW,index_col)

    else:
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,index_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col)



    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    new_copy['sites_code'] = [re.sub('_[0-9]$', '', str(x)) for x in new_copy['sites']]
    new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')
    
    """n = 4
    lista = []
    for site in new_copy['sites']:
        chunks = [site[i:i+n] for i in range(0, len(site), n)]
        lista.append(chunks[0])
    new_copy['sites_code'] = lista
    new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]"""

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    ws.delete_cols(2)
    wb.save(fname)
    print('\nDone.\n')

pathta = '/content/TA_Input_Hungary_20210831.csv'
pathold = '/content/TA_Input_Hungary_20210731.csv'
ta_save = '/content/HU_TA'
ta_on = '20210731.csv'
ta_nn = '20210831.csv'
ta_bill = ['SiteID', 'MNO classification','Base fee (@ signing) in HUF', 'Starting date', 'Expiring date (raw data)']
#path_OLD, path_NEW, index_col, bill_cols, path_save, old_name, new_name, type_file='mix',kind='tw'
find_diffs_between_files(pathold, pathta, 'SiteID', ta_bill, ta_save,ta_on,ta_nn, 'csv', kind='ta')


New Rows:     ['1077digi távközlési és szolgáltató kft.mno 3', '1253digi távközlési és szolgáltató kft.mno 3', '1781digi távközlési és szolgáltató kft.mno 3', '1866digi távközlési és szolgáltató kft.mno 3', '2036digi távközlési és szolgáltató kft.mno 3', '2362digi távközlési és szolgáltató kft.mno 3', '3094digi távközlési és szolgáltató kft.mno 3', '4089wavecom informatikai kft.otmo', '4095wavecom informatikai kft.otmo', '4821wavecom informatikai kft.otmo', '4906wavecom informatikai kft.otmo', '5048digi távközlési és szolgáltató kft.mno 3', '5870digi távközlési és szolgáltató kft.mno 3', '6009digi távközlési és szolgáltató kft.mno 3']
Dropped Rows: ['4089wavecom informatikai kftotmo', '4095wavecom informatikai kftotmo', '4821wavecom informatikai kftotmo', '4906wavecom informatikai kftotmo']

Done.



LC Comparions

In [ ]:
import natsort
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name, new_name, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col):
        col = lower_str(list(df.columns))
        df.columns = col
        #df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        #df = df.drop_duplicates(subset=[index_col], keep='last')
        #df[index_col] = df[index_col].astype(str)
        #df['sites'] = df[index_col]
        #df = df.iloc[natsort.index_humansorted(df[index_col])]
        df = df.sort_values(by=[index_col, 'counterpart'])
        lista=[]
        dico = {}
        for id in df[index_col]:
            nome = str(id)
            if nome not in dico.keys():
                dico[nome] = 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)
            else:
                dico[nome] += 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)
        print(lista)
        df['sites'] = lista
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    #status_col = status_col.lower()
    
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD,index_col)

        df_NEW = pd.read_csv(path_NEW, encoding='latin1').fillna('')
        df_NEW = fit_df(df_NEW,index_col)

    else:
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,index_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col)



    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')
    
    n = 4
    lista = []
    for site in new_copy['sites']:
        chunks = [site[i:i+n] for i in range(0, len(site), n)]
        lista.append(chunks[0])
    new_copy['sites_code'] = lista
    new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

pathlc = '/content/LC_Input_Hungary_20210831.csv'
patholdlc = '/content/LC_Input_Hungary_20210731.csv'
lc_save = '/content/HU_LC'
lc_on = '20210731.csv'
lc_nn = '20210831.csv'
lc_bill = ['Code', 'Current annual lease fees in HUF','Starting date', 'Expiring date (raw data)']
#path_OLD, path_NEW, index_col, bill_cols, path_save, old_name, new_name, type_file='mix',kind='tw'
find_diffs_between_files(patholdlc, pathlc, 'Code', lc_bill, lc_save,lc_on,lc_nn, 'csv', kind='ta')

UIS Comparisons

In [ ]:
def find_diffs_between_files_uis(path_OLD, path_NEW, index_col, uis_cols, \
                             path_save, old_name, new_name, type_file='mix', sheetname='', skipr=0, skipc=0):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, uis_cols):

        df = df.dropna(subset=[index_col], axis=0)
        #df = df[uis_cols]
        #df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    #uis_cols = lower_str(uis_cols)
    #index_col = index_col.lower()
    # type_file = type_file.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD, sheet_name = sheetname,  skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = df_OLD.drop([0,0])
        df_OLD = fit_df(df_OLD,uis_cols)


        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname,  skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = df_NEW.drop([0,0]) 
        df_NEW = fit_df(df_NEW,uis_cols)


    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,uis_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,uis_cols)

    else:
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,uis_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,uis_cols)

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))
    print(sharedCols)
    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0,thresh=len(new_copy.columns[1:]))
    new_copy = new_copy.dropna(axis=1)

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - [{old_name}] vs [{new_name}].xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ1000', highlight)
    
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

col_uis = ['Site_ID (Alphanumeric, Alphabetical or Numeric)',
       'Site Categorization as Phase 1 or Phase 2 Site',
       'BTS site applicable charge (Annual)',
       'Commercials for sites beyond 10% cap of critical sites (Annual)',
       'Total charges to be applied for subsequent active sharing arrangement',
       'Fixed component of energy bill', 'Variable component of energy bill',
       'Energy consumption (for metered site)',
       'Agreed unit price (for metered sites)',
       'Operator’s active energy charge through estimated model',
       'Capex incurred by TowerCo under clause 29 to be reimbursed by Operator',
       'Energy reduction measures - Final value to be charged',
       'Legacy site upgrades Or BBU capacity upgrade - Final value to be charged',
       'BTS site - Site service order withdrawal - Final value to be charged',
       'Energy infrastructure enhancements - Final value to be charged',
       'One-off power related costs - Final value to be charged',
       'Space blocking - Unutilized sites\nIf unutilized then select Yes ',
       'Decommissioning charges - Equipment removal cost - Final value to be charged',
       'Site exit fee - Site ID part of delta',
       'Site exit fee - Remaining term of site IDs in delta',
       'Site power availability - Site IDs where service credits are applicable',
       'Site power availability - Total hours applicable ',
       'Site power availability - Unavailable time',
       'Site power availability - Count of repeat incidents (if applicable)',
       'Site cooling - Final value associated with Service Credit applicable',
       'Site access - Final value associated with Service Credit applicable',
       'Incident resolution - Final value associated with Service Credit applicable',
       'BTS sites - Deployment time - Delay',
       'Site modification Completion time - Delay',
       'Delay In RSCR site modification completion Time',
       'Delay In RSCR BTS site completion time',
       'Additional Customer Discount percentage applicable (Number only)',
       'Delay in Site Modification Projects', 'Delay in BTS Projects',
       'Excess of Upgrade Capital Expenditure over Threshold']
       
pathuis_n = '/content/UserInput_Hungary_20210831.xlsx'
pathuis_o = '/content/UserInput_Hungary_20210630.xlsx'
sheetuis='SiteLevel'
skiprows=2
to_save = '/content/HU_UIS'

old_n = 'UIS - 20210630_true_up.xlsx'
new_n = 'UIS - 20210831.xlsx'

find_diffs_between_files_uis(pathuis_o, pathuis_n, 'Site_ID (Alphanumeric or Numeric)', col_uis, \
                        to_save, old_n, new_n, type_file='excel', sheetname=sheetuis, skipr=2)